**Sequence to Sequence Learning with Neural Networks (NIPS 2014) 실습**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torchtext==0.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.0
    Uninstalling torchtext-0.16.0:
      Successfully uninstalled torchtext-0.16.0


#### **데이터 전처리(Preprocessing)**
- spaCy 라이브러리 : 문장의 토큰화, 태깅 등 전처리 기능을 위한 라이브러리

In [3]:
!python -m spacy download en
!python -m spacy download de

2024-01-09 11:28:04.542875: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-09 11:28:04.542932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-09 11:28:04.546811: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-09 11:28:04.557846: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-09 11:28:06.442590: W tensorflow/compiler/tf2

In [4]:
import spacy

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화
spacy_de = spacy.load('de_core_news_sm') # 독일어 토큰화

In [5]:
# 간단한 토큰화 예제
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


In [6]:
# 독일어 문장을 토큰화한 후 순서를 뒤집는 함수
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)][::-1]

# 영어 문장을 토큰화하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

- Field 라이브러리를 이용해 데이터셋에 대한 전처리 내용을 명시
- SRC : 독일어, TRC : 영어

In [7]:
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True)

In [8]:
from torchtext.datasets import Multi30k

train_data, valid_data, test_data = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG), root="/content/drive/MyDrive/Paper Review Code/data")

In [9]:
print(f"train data 크기: {len(train_data.examples)}개")
print(f"valid data 크기: {len(valid_data.examples)}개")
print(f"test data 크기: {len(test_data.examples)}개")

train data 크기: 29000개
valid data 크기: 1014개
test data 크기: 1000개


In [10]:
# train data에서 하나를 선택해 출력
print(vars(train_data.examples[30])['src'])
print(vars(train_data.examples[30])['trg'])

['.', 'steht', 'urinal', 'einem', 'an', 'kaffee', 'tasse', 'einer', 'mit', 'der', ',', 'mann', 'ein']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


- Field 객체의 build_vocab 메서드를 이용해 영어와 독일어의 단어 사전을 생성
  - 최소 2번 이상 등장한 단어만 선택

In [11]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

print(f"len(SRG): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRG): 7857
len(TRG): 5894


In [12]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어 : 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩 : 1
print(TRG.vocab.stoi["<sos>"]) # <sos> : 2
print(TRG.vocab.stoi["<eos>"]) # <eos> : 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4115
1754


- 한 문장에 포함된 단어가 연속적으로 LSTM에 입력되어야 함
  - 따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋음
  - 이를 위해 BucketIterator를 사용
  - BucketIterator는 PyTorch의 torchtext 라이브러리에서 제공하는 데이터 로더 중 하나
  - 시퀀스 길이에 따라 데이터를 정렬하고 비슷한 길이의 시퀀스들을 함께 묶어주는 역할
  - 배치 크기 : 128


In [13]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 128

# 일반적인 데이터 로더의 iterator와 유사하게 사용 가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    device=device)

In [14]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg
    print(f"첫 번째 배치 크기: {src.shape}")

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[0]):
        print(f"인덱스 {i}: {src[i][0].item()}")

    # 첫 번째 배치만 확인
    break

첫 번째 배치 크기: torch.Size([30, 128])
인덱스 0: 2
인덱스 1: 4
인덱스 2: 63
인덱스 3: 39
인덱스 4: 12
인덱스 5: 149
인덱스 6: 10
인덱스 7: 1802
인덱스 8: 6
인덱스 9: 12
인덱스 10: 31
인덱스 11: 1125
인덱스 12: 46
인덱스 13: 14
인덱스 14: 11
인덱스 15: 26
인덱스 16: 70
인덱스 17: 5
인덱스 18: 3
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1
인덱스 25: 1
인덱스 26: 1
인덱스 27: 1
인덱스 28: 1
인덱스 29: 1


#### **인코더 아키텍처**
- 주어진 소스 문장을 context vector로 인코딩
- LSTM은 hidden state와 cell state를 반환
- 하이퍼 파라미터
  - input_dim : 하나의 단어에 대한 원핫 인코딩 차원
  - embed_dim : 임베딩 차원
  - hidden_dim : 히든 상태 차원
  - n_layers : RNN 레이어의 개수
  - dropout_ratio : 드롭아웃 비율

In [15]:
import torch.nn as nn

# 인코더 아키텍처 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩은 원-핫 인코딩을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

        # 드롭아웃
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더는 소스 문장을 입력으로 받아 context vector를 반환
    def forward(self, src):
        # src : [단어 개수, 배치 크기]
        embedded = self.dropout(self.embedding(src))
        # embedded : [단어 개수, 배치 크기, 임베딩 차원]

        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs : [단어 개수, 배치 크기, 히든 차원] => 현재 단어의 출력 정보
        # hidden : [레이어 개수, 배치 크기, 히든 차원] => 현재까지의 모든 단어의 정보
        # cell : [레이어 개수, 배치 크기, 히든 차원] => 현재까지의 모든 단어의 정보

        # context vector 반환
        return hidden, cell

#### **디코더 아키텍처**
- 주어진 context vector를 타겟 문장으로 디코딩
- LSTM은 hidden state와 cell state를 반환
- 하이퍼 파라미터
  - input_dim : 하나의 단어에 대한 원-핫 인코딩 차원
  - embed_dim : 임베딩 차원
  - hidden_dim : 히든 상태 차원
  - n_layers : RNN 레이어 개수
  - dropout_ratio : 드롭아웃 비율

In [16]:
# 디코더 아키텍처 정의
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩은 원-핫 인코딩 말고 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(output_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

        # FC 레이어 (인코더와 다른 부분)
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        # 드롭아웃
        self.dropout = nn.Dropout(dropout_ratio)

    # 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장을 반환
    def forward(self, input, hidden, cell):
        # input : [배치 크기] => 단어의 개수는 항상 1개
        # hidden : [레이어 개수, 배치 크기, 히든 차원]
        # cell : [레이어 개수, 배치 크기, 히든 차원]
        input = input.unsqueeze(0)
        # input : [단어 개수 = 1, 배치 크기]

        embedded = self.dropout(self.embedding(input))
        # embedded : [단어 개수, 배치 크기, 임베딩 차원]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output : [단어 개수 = 1, 배치 크기, 히든 차원] => 현재 단어의 출력 정보
        # hidden : [레이어 개수, 배치 크기, 히든 차원] => 현재까지의 모든 단어 정보
        # cell : [레이어 개수, 배치 크기, 히든 차원] => 현재까지의 모든 단어 정보

        # 단어 개수는 1개이므로 차원 제거
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [배치 크기, 출력 차원]

        # (현재 출력 단어, 현재까지의 모든 단어 정보, 현재까지의 모든 단어 정보)
        return prediction, hidden, cell

#### Seq2Seq 아키텍처
- 앞서 정의한 인코더와 디코더를 가지고 있는 하나의 아키텍처
  - 인코더 : 주어진 문장을 context vector로 인코딩
  - 디코더 : 주어진 context vector를 타겟 문장으로 디코딩
  - 단, 디코더는 한 단어씩 넣어서 한 번씩 결과를 구함

- Teacher forcing : 디코더의 예측을 다음 입력으로 사용하지 않고, 실제 목표 출력을 다음 입력으로 사용하는 기법

In [17]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    # 학습할 때는 완전한 형태의 소스 문장, 타겟 문장, teacher_forcing_ratio를 넣기
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src : [단어 개수, 배치 크기]
        # trg : [단어 개수, 배치 크기]
        # 먼저 인코더를 거쳐 context vector를 추출
        hidden, cell = self.encoder(src)

        # 디코더의 최종 결과를 담을 텐서 객체 만들기
        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[1] # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # 첫 번째 입력은 항상 <sos> 토큰
        input = trg[0, :]

        # 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output # FC를 거쳐서 나온 현재의 출력 단어 정보
            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

            # teacher_forcing_ratio : 학습할 때 실제 목표 출력(ground-truth)을 사용하는 비율
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기

        return outputs

#### **학습(Training)**

In [18]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
encoder_embed_dim = 256
decoder_embed_dim = 256
hidden_dim = 512
n_layers = 2
enc_dropout_ratio = 0.5
dec_dropout_ratio = 0.5

In [19]:
# 인코더와 디코더 객체 선언
enc = Encoder(input_dim, encoder_embed_dim, hidden_dim, n_layers, enc_dropout_ratio)
dec = Decoder(output_dim, decoder_embed_dim, hidden_dim, n_layers, dec_dropout_ratio)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

- 논문에서 설정한 것과 같이 $\mathcal{U}(-0.08, 0.08)$의 값으로 모델 가중치 파라미터 초기화

In [20]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7857, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5894, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5894, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

- 학습 및 평가 함수 정의

In [21]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [22]:
# 모델 학습 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    # 전체 학습 데이터를 확인
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)
        # output : [출력 단어 개수, 배치 크기, 출력 차원]
        ouput_dim = output.shape[-1]

        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        # output = [(출력 단어의 개수 - 1) * batch size, output dim]
        trg = trg[1:].view(-1)
        # trg = [(타겟 단어의 개수 - 1) * batch size]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 함수 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기 clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [23]:
# 모델 평가 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 평가할 때 teacher forcing는 사용하지 않음
            output = model(src, trg, 0)
            # output: [출력 단어 개수, 배치 크기, 출력 차원]
            output_dim = output.shape[-1]

            # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output = [(출력 단어의 개수 - 1) * batch size, output dim]
            trg = trg[1:].view(-1)
            # trg = [(타겟 단어의 개수 - 1) * batch size]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [24]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
import time
import math
import random

N_EPOCHS = 20
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 0m 45s
	Train Loss: 5.067 | Train PPL: 158.644
	Validation Loss: 5.015 | Validation PPL: 150.612
Epoch: 02 | Time: 0m 42s
	Train Loss: 4.522 | Train PPL: 92.007
	Validation Loss: 4.811 | Validation PPL: 122.812
Epoch: 03 | Time: 0m 43s
	Train Loss: 4.213 | Train PPL: 67.557
	Validation Loss: 4.545 | Validation PPL: 94.197
Epoch: 04 | Time: 0m 43s
	Train Loss: 3.997 | Train PPL: 54.461
	Validation Loss: 4.489 | Validation PPL: 89.076
Epoch: 05 | Time: 0m 43s
	Train Loss: 3.843 | Train PPL: 46.652
	Validation Loss: 4.358 | Validation PPL: 78.133
Epoch: 06 | Time: 0m 43s
	Train Loss: 3.740 | Train PPL: 42.112
	Validation Loss: 4.339 | Validation PPL: 76.628
Epoch: 07 | Time: 0m 43s
	Train Loss: 3.609 | Train PPL: 36.911
	Validation Loss: 4.267 | Validation PPL: 71.286
Epoch: 08 | Time: 0m 43s
	Train Loss: 3.455 | Train PPL: 31.658
	Validation Loss: 4.198 | Validation PPL: 66.574
Epoch: 09 | Time: 0m 43s
	Train Loss: 3.343 | Train PPL: 28.293
	Validation Loss: 4.095 | Val

In [27]:
# 학습된 모델 저장
from google.colab import files

files.download('seq2seq.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Paper Review Code/data/seq2seq.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

Test Loss: 3.714 | Test PPL: 41.019


In [29]:
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더(endocer)에 소스 문장을 넣어 문맥 벡터(context vector) 계산
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

In [30]:
example_idx = 10

src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['.', 'freien', 'im', 'tag', 'schönen', 'einen', 'genießen', 'sohn', 'kleiner', 'ihr', 'und', 'mutter', 'eine']
타겟 문장: ['a', 'mother', 'and', 'her', 'young', 'song', 'enjoying', 'a', 'beautiful', 'day', 'outside', '.']
전체 소스 토큰: ['<sos>', '.', 'freien', 'im', 'tag', 'schönen', 'einen', 'genießen', 'sohn', 'kleiner', 'ihr', 'und', 'mutter', 'eine', '<eos>']
소스 문장 인덱스: [2, 4, 89, 20, 203, 779, 19, 566, 625, 70, 137, 10, 365, 8, 3]
모델 출력 결과: a father and father are enjoying a on a public day . <eos>


In [31]:
src = tokenize_de("Guten Abend.")

print(f'소스 문장: {src}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['.', 'Abend', 'Guten']
전체 소스 토큰: ['<sos>', '.', 'abend', 'guten', '<eos>']
소스 문장 인덱스: [2, 4, 1162, 3801, 3]
모델 출력 결과: tortillas . <eos>
